# 모바일 액세서리 분석

In [1]:
#데이터셋 가져오기
#order 컬럼명 바꾸기 - 숫자로 되어 있는 부분에 Order (purchase event) 입력
import pandas as pd
#카테고리 전처리
import re
# 장바구니 분석
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
import os
from collections import Counter

In [2]:
input_path = 'C:/GMG_R&D/3. 장바구니 분석/1. 한총' #input("작업 파일 경로를 입력하세요")
os.chdir(input_path)
path=os.getcwd()
print(path)

#폴더 생성
#x = dt.datetime.now()

file_name=f'모바일 액세서리 조합 결과'

if not os.path.exists(file_name):
    os.mkdir(file_name) 

C:\GMG_R&D\3. 장바구니 분석\1. 한총


### 1. 한총 데이터 로드

In [3]:
df= pd.read_clipboard(sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50001 entries, Model Code (v41) to EF-RS906CNEGKR,EP-TA800XWKGKR
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   2020년            50001 non-null  int64
 1   2021년            50001 non-null  int64
 2   2022.01~2022.11  50001 non-null  int64
dtypes: int64(3)
memory usage: 1.5+ MB


In [4]:
df=df[1:]
df=df.reset_index().rename(columns={"index": "Model Code (v41)"})
df['2020.01~2022.11']=df.iloc[:,1:4].sum(axis=1)
df

,Model Code (v41),2020년,2021년,2022.01~2022.11,2020.01~2022.11
0,EP-DG930MBKGKR,9746,6236,3665,19647
1,AA-BC7AU5B,9365,0,0,9365
2,SM-R175NZWAKOO,4961,344,0,5305
3,SM-N986NZNEKOO,4950,172,0,5122
4,SM-R175NZBAKOO,4906,127,0,5033
...,...,...,...,...,...
49995,"EJ-P5450SBEGKR,GP-TTT870KDATK,GP-TTT870KDATK",0,0,1,1
49996,"1FF10014,1FF10014,1FF10014,SAM20220221001,SAM2...",0,0,1,1
49997,"V00073,V00225,V00235",0,0,1,1
49998,"21,405,702,145,135",0,0,1,1


### 2. 카테고리 ( 모바일 관련 카테고리만 가져오기 )

In [5]:
#카테고리 가져오기
rex= pd.read_clipboard(sep='\t')
rex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Category1D        5 non-null      object
 1   Category2D        5 non-null      object
 2   Category2D_Value  5 non-null      object
dtypes: object(3)
memory usage: 248.0+ bytes


### 1) 비중

In [9]:

new_df=[]

for mobile in df.columns[1:]:
    print(f'************< {mobile} >******************')
    
    all=df[mobile].sum()
    one=df[~df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()
    two=df[df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()
    percent_one=round((df[~df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()/all)*100,2)
    percent_two=round((df[df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')][mobile].sum()/all)*100,2)

    
    
    print('전체: ',all)
    print('1개: ',one)
    print('2개이상: ',two)
    print("제품 1개 구매 비중: ",percent_one)
    print("제품 2개이상 구매 비중: ",percent_two)

    new_df1=[]
    for i in [all, one, two, percent_one,percent_two]:
        new_df1.append(i)
    new_df.append(new_df1)

# 데이터 저장3: 데이터 기준
pd.DataFrame(new_df,columns=['전체','1개','2개이상','제품 1개 구매 비중', '제품 2개이상 구매 비중'],
            index=[df.columns[1],df.columns[2],df.columns[3],df.columns[4]]).to_csv(f'{path}/{file_name}/[최종]기초통계량결과.csv', encoding='utf-8-sig')
pd.DataFrame(new_df,columns=['전체','1개','2개이상','제품 1개 구매 비중', '제품 2개이상 구매 비중'])

************< 2020년 >******************
전체:  365380
1개:  301555
2개이상:  63825
제품 1개 구매 비중:  82.53
제품 2개이상 구매 비중:  17.47
************< 2021년 >******************
전체:  738544
1개:  656707
2개이상:  81837
제품 1개 구매 비중:  88.92
제품 2개이상 구매 비중:  11.08
************< 2022.01~2022.11 >******************
전체:  842619
1개:  784183
2개이상:  58436
제품 1개 구매 비중:  93.06
제품 2개이상 구매 비중:  6.94
************< 2020.01~2022.11 >******************
전체:  1946543
1개:  1742445
2개이상:  204098
제품 1개 구매 비중:  89.51
제품 2개이상 구매 비중:  10.49


,전체,1개,2개이상,제품 1개 구매 비중,제품 2개이상 구매 비중
0,365380,301555,63825,82.53,17.47
1,738544,656707,81837,88.92,11.08
2,842619,784183,58436,93.06,6.94
3,1946543,1742445,204098,89.51,10.49


### 모바일 액세서리 분석

In [6]:
for mobile in df.columns[1:]:
    print(f'************< {mobile} >******************')
    
    mobile_df=df[df[mobile]!= 0]
    #print('order 0제외 행개수:',mobile_df.shape[0])
    mobile_df_1=mobile_df[~mobile_df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')]
    mobile_df_2=mobile_df[mobile_df[['Model Code (v41)',mobile]]['Model Code (v41)'].str.contains(',')]
    
    code_book={'2개이상':mobile_df_2}
    list_a=[]
    for key, value in code_book.items():
        value=value.reset_index(drop=True)
        data=value
        a = data['Model Code (v41)'].tolist()
        b = ','.join(a) #리스트 문자열로
        c = b.split(',') #문자열 개별 문자로 리스트화 
        model = list(set(c)) #중복 제거

        #전체 SKU 담을 딕셔너리
        dic_category = {}

        for m in model :
            flag = 1
            for i in range(len(rex)):
                r = re.compile(rex.Category2D_Value[i])
                if r.search(m):
                    dic_category[m] = rex.Category1D[i] +'_'+ rex.Category2D[i]  #rex.Category1D[i] +'_'+  #sku 규칙을 알고자한다면, r.search(m).group()
                    flag = 0
                    break
            if flag :
                dic_category[m] = '기타'

        #데이터셋 만들기
        dataset = []    # 데이터셋 구조 [ [a,b], [a,c], [a,b,c] .... ]

        for i in range(len(data)) :
            sku = data['Model Code (v41)'][i]
            revised = sku.split(",")
            code = []
            for j in range(len(revised)) :
                code.append(dic_category[revised[j]])   #각 SKU 리스트에 담기
            
            for k in range(data[mobile][i]) :   #order 수만큼 담기
                dataset.append(code) 

        print('기존 dataset 행 수: ',len(dataset))

        #기타제거
        ho = []
        rm_set = {'기타'}
        for s in dataset:
            arr_new = [i for i in s if i not in rm_set]
            ho.append(arr_new) # [5, 7, 2, 7]
        print('기타 제거 후',len(ho))
        dataset=ho

        category1='모바일'
        select_mobile1=[s for s in dataset if any(category1 in l for l in s)]
   
        category='모바일 액세서리'
        select_mobile=[s for s in select_mobile1 if any(category in l for l in s)]

        print('모바일 액세서리가 포함된 행수: ',len(select_mobile))    #빈칸 제거 및 모바일 액세서리 아닌 것 제외
        
        
         
        #실제데이터중 UNIQE 값으로
        count=Counter([tuple(sorted(i)) for i in sorted([tuple(set(item)) for item in dataset])])
        da=sorted(count.items(), key=lambda i: i[1], reverse=True)
        
        pd.DataFrame(da).to_csv(f'{path}/{file_name}/조합 유니크 결과_{mobile}_{category}.csv', index=False, encoding='utf-8-sig')
       
       

************< 2020년 >******************
기존 dataset 행 수:  63825
기타 제거 후 63825
모바일 액세서리가 포함된 행수:  54885
************< 2021년 >******************
기존 dataset 행 수:  81837
기타 제거 후 81837
모바일 액세서리가 포함된 행수:  63923
************< 2022.01~2022.11 >******************
기존 dataset 행 수:  58436
기타 제거 후 58436
모바일 액세서리가 포함된 행수:  23117
************< 2020.01~2022.11 >******************
기존 dataset 행 수:  204098
기타 제거 후 204098
모바일 액세서리가 포함된 행수:  141925
